In [0]:
import requests
import os
import zipfile
from datetime import datetime

In [0]:
base_url = "https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/servidores/"

def download_file(year, month):

    month_str = str(month).zfill(2)
    file_name = f"{year}{month_str}_Servidores_SIAPE.zip"
    file_url = os.path.join(base_url, file_name)
    local_path = os.path.join("/dbfs/servidores/", file_name)
    
    if not os.path.exists("/dbfs/servidores/"):
        os.mkdir("/dbfs/servidores/")

    # Olha se já existe, se não, baixa
    if not os.path.exists(local_path):
        print(f"Baixando: {file_name}...")
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(response.content)
            print(f"Download concluído: {file_name}")
        else:
            print(f"Erro ao baixar o arquivo: {response.status_code}")
    else:
        print(f"Arquivo já existe: {file_name}")

current_date = datetime.now()
current_month = current_date.month
current_year = current_date.year

historical_months = []
for i in range(4):
    month = current_month - (2 + i)
    year = current_year

    if month < 1:
        month += 12
        year -= 1

    historical_months.append((year, month))

# Baixar os arquivos para os meses determinados
for year, month in historical_months:
    download_file(year, month)


Baixando: 202408_Servidores_SIAPE.zip...
Download concluído: 202408_Servidores_SIAPE.zip
Baixando: 202407_Servidores_SIAPE.zip...
Download concluído: 202407_Servidores_SIAPE.zip
Baixando: 202406_Servidores_SIAPE.zip...
Download concluído: 202406_Servidores_SIAPE.zip
Baixando: 202405_Servidores_SIAPE.zip...
Download concluído: 202405_Servidores_SIAPE.zip


In [0]:
def unzip_files(historical_months):
    for year, month in historical_months:
        month_str = str(month).zfill(2)
        file_name = f"{year}{month_str}_Servidores_SIAPE.zip"
        local_path = os.path.join("/dbfs/servidores/", file_name)
        base_extract_path = "/dbfs/servidores_unzip_raw/"

        if os.path.exists(local_path):
            with zipfile.ZipFile(local_path, 'r') as zip_ref:
                # Extrair os arquivos do zip e organizá-los por tipo de dado
                for file in zip_ref.namelist():
                    # Identificar tipo de dado a partir do nome do arquivo
                    if "Remuneracao" in file:
                        target_path = os.path.join(base_extract_path, "remuneracao")
                    elif "Observacoes" in file:
                        target_path = os.path.join(base_extract_path, "observacoes")
                    elif "Cadastro" in file:
                        target_path = os.path.join(base_extract_path, "cadastro")
                    elif "Afastamento" in file:
                        target_path = os.path.join(base_extract_path, "afastamento")
                    else:
                        continue
                    
                    os.makedirs(target_path, exist_ok=True)
                    zip_ref.extract(file, target_path)
            
            print(f"Arquivo descompactado: {file_name}")
        else:
            print(f"Arquivo não encontrado para descompactar: {file_name}")

# Descompactar os arquivos
unzip_files(historical_months)

Arquivo descompactado: 202408_Servidores_SIAPE.zip
Arquivo descompactado: 202407_Servidores_SIAPE.zip
Arquivo descompactado: 202406_Servidores_SIAPE.zip
Arquivo descompactado: 202405_Servidores_SIAPE.zip
